An algorithm that simulates the resting of an infinite plane on top of a diffuse set of points. It will find the final position + orientation of the plane such that the plane hasn't passed through any points but rests on 3+ of the points. Rough strategy:
1. Push plane towards point cloud until it collides with first point (p1).
2. Rotate the plane around p1 as a pivot the minimum distance until it is touching a second point (p2).
3. Rotate the plane around the line between p1 and p2 until it is touching a 3rd point.


In [4]:
import numpy as np

In [5]:
def to_homo(vec):
    output = np.ones(4)
    output[:3] = vec
    return output

def update_plane(center_pt, normal, T):
    new_normal = np.dot(T[:3, :3], normal)
    new_center = np.dot(T, to_homo(center_pt))[:3]
    return new_center, new_normal

# Implementation of the Rodriguez formula
def rotation_matrix(axis, theta):

    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

# Generates random vector that is orthogonal to given vector
def orthogonal_vec(vec1):
    vec1_norm = vec1 / np.linalg.norm(vec1)
    while(True):
        vec2 = np.random.random(3)
        vec2 = vec2 - np.dot(vec2, vec1_norm) * vec1_norm
        if np.linalg.norm(vec2) > 0:
            break
    
    print('fff', np.dot(vec2, vec1))
    return vec2 / np.linalg.norm(vec2)

In [6]:
def fit_plane(center, normal, pts, dist_cutoff=1e-10):
    assert len(pts) >= 3
    transforms = []
    cur_center = center.copy()
    cur_normal = normal.copy()

    normal_dists = np.dot(pts - cur_center.reshape(1, -1), cur_normal)
    points_on_plane = pts[normal_dists <= dist_cutoff]
    other_points = pts[normal_dists > dist_cutoff]
    while True:
        num_matched = len(points_on_plane)
        if not num_matched: 
            # Step 1: Move along normal until first collision
            T = np.eye(4)
            T[:3, 3] = normal * np.min(normal_dists)
        elif  num_matched == 1:
            # Step 2: Find point the requires smallest rotation to get into plane and then rotate
            on_plane_pt = points_on_plane[0]
            
            # Project points into plane
            out_of_plane_vecs = other_points - on_plane_pt.reshape(1, -1)
            out_of_plane_vecs = out_of_plane_vecs / np.linalg.norm(out_of_plane_vecs, axis=1).reshape(-1, 1)
            normal_components = np.dot(out_of_plane_vecs, cur_normal).reshape(-1, 1)
            projected_vecs = out_of_plane_vecs - cur_normal * normal_components
            
            # The vector with the longest projection will require the smallest rotation to put it in plane
            min_rot_idx = np.argmax(np.linalg.norm(projected_vecs.reshape(-1, 3), axis=1))
            rot_vec_1 = out_of_plane_vecs[min_rot_idx]
            rot_vec_2 = projected_vecs[min_rot_idx]
            
            if np.linalg.norm(rot_vec_2) <= dist_cutoff:
                # Degenerate case (all points are along the normal)
                rot_angle = np.pi / 2
                axis_of_rot = orthogonal_vec(normal)
            else:
                # Normal case
                rot_vec_2 = rot_vec_2 / np.linalg.norm(rot_vec_2)
                axis_of_rot = np.cross(rot_vec_2, rot_vec_1)  # We are rotating the plane into the point
                axis_of_rot = axis_of_rot / np.linalg.norm(axis_of_rot)
            
            # Generate transform
            rot_angle = np.arccos(np.dot(rot_vec_1, rot_vec_2))
            rot_matrix = rotation_matrix(axis_of_rot, rot_angle)
            trans = on_plane_pt - np.dot(rot_matrix, on_plane_pt)
            T = np.eye(4)
            T[:3, :3] = rot_matrix
            T[:3, 3] = trans       
        if num_matched == 2: 
            # Step 3: Rotating about axis between two on-plane points, find the point that requires smallest rot
            on_plane_pt1 = points_on_plane[0]
            on_plane_pt2 = points_on_plane[1]
            axis_of_rot = on_plane_pt2 - on_plane_pt1
            axis_of_rot = axis_of_rot / np.linalg.norm(axis_of_rot)
            
            # Project the vectors into the plane perpandicular to the axis of rotation
            out_of_plane_vecs = other_points - on_plane_pt1.reshape(1, -1)
            rot_axis_normal_components = np.dot(out_of_plane_vecs, axis_of_rot).reshape(-1, 1)   
            on_line_vecs = out_of_plane_vecs - axis_of_rot * rot_axis_normal_components
            on_line_vecs = on_line_vecs / np.linalg.norm(on_line_vecs, axis=1).reshape(-1, 1)
            plane_normal_components = np.dot(on_line_vecs, cur_normal).reshape(-1, 1)
            projected_vecs = on_line_vecs - cur_normal * plane_normal_components
            min_rot_idx = np.argmax(np.linalg.norm(projected_vecs.reshape(-1, 3), axis=1))
            rot_vec_1 = on_line_vecs[min_rot_idx]
            rot_vec_1 = rot_vec_1 / np.linalg.norm(rot_vec_1)
            rot_vec_2 = projected_vecs[min_rot_idx]
            
            if np.linalg.norm(rot_vec_2) <= dist_cutoff:
                # Degenerate case (3rd point directly 'beneath' line)
                rot_angle = np.pi / 2
                rot_sign = 1
            else:
                # Normal case
                rot_vec_2 = rot_vec_2 / np.linalg.norm(rot_vec_2)
                rot_angle = np.arccos(np.dot(rot_vec_1, rot_vec_2))
                rot_sign = np.sign(np.dot(axis_of_rot, np.cross(rot_vec_2, rot_vec_1)))
            # Generate transform
            rot_matrix = rotation_matrix(axis_of_rot, rot_sign * rot_angle)
            trans = on_plane_pt2 - np.dot(rot_matrix, on_plane_pt2)
            T = np.eye(4)
            T[:3, :3] = rot_matrix
            T[:3, 3] = trans   
        elif num_matched > 2:
            break
        transforms.append(T)
        cur_center, cur_normal = update_plane(cur_center, cur_normal, T)
        normal_dists = np.dot(pts - cur_center.reshape(1, -1), cur_normal)
        points_on_plane = pts[normal_dists <= dist_cutoff]
        other_points = pts[normal_dists > dist_cutoff]

    T_final = np.eye(4)
    for T in transforms:
        T_final = np.dot(T, T_final)
    
    return T_final, cur_center, cur_normal, normal_dists

In [11]:
def test(n_points):
    pts = np.random.random((n_points,3))
    T, fin_cent, fin_normal, dists = fit_plane(center, normal, pts)
    assert (np.sum(dists < dist_cutoff) == 3) and (np.sum(dists < -dist_cutoff) == 0), "Test Failed"
    
N = 50000
for _ in range(10):
    test(N)